In [61]:
import requests
import re

import pandas as pd
df = pd.DataFrame(columns=['source', 'news', 'linc', 'date'])

import pymongo as pm
from user_agent import generate_user_agent

from lxml import html

In [109]:
def get_atr_mailru(u):
    global df
    req = requests.get(u, headers = {"User-Agent":generate_user_agent()}).text
    root = html.fromstring(req)
    
    source = root.xpath("//span[contains(text(),'{}')]".format('"'))
    source = source if len(source) > 0 else root.xpath("//a[@class='link color_gray breadcrumbs__link']//span[@class='link__text']")
    source = source[0].text if len(source) > 0 else 'не подписано'
    news = root.xpath("//h1[@class='hdr__inner']")[0].text
    linc = u
    date = root.xpath("//span[@class='note__text breadcrumbs__text js-ago']/@datetime | //time[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    
    return [source,news,linc,date]
    

def get_news_mailru():
    global df
    url = 'https://mail.ru/?from=m'
    req = requests.get(url, headers = {"User-Agent":generate_user_agent()}).text
    
    root = html.fromstring(req)

    for item in root.xpath("//div[@class='list'][1]/a/@href"):
        print(item)
        df.loc[len(df)+1] = get_atr_mailru(item)
    print(df)

In [293]:
def get_atr_lentaru(u):
    global df
    print(u)
    req = requests.get(u, headers = {"User-Agent":generate_user_agent()}).text
    root = html.fromstring(req)
    
    source = 'lenta.ru'
    news = root.xpath("//h1[@class='b-topic__title'] | //div[@class='premial-header__title'] | //h1[@class='brief-header__title']")[0].text
    linc = u
    try:
        date = root.xpath("//div[@class='b-topic__info']/time/@datetime | //div[@class='premial-header__date']")[0]
    except:
        date = "не определена"
    
    return [source,news,linc,date]
    

def get_news_lentaru():
    global df
    url = 'https://lenta.ru'
    req = requests.get(url, headers = {"User-Agent":generate_user_agent()}).text

    root = html.fromstring(req)

    for item in root.xpath("//section[@class='row b-top7-for-main js-top-seven']//div[@class='item']/a/@href | //section[@class='b-layout js-layout b-layout_main']//div[@class='titles']//a/@href"):
        ext = re.findall('(https:\/\/[\w\. _\d]+)\/|\/(extlink)\/', item)

        if len(ext) > 0:
            r1 = root.xpath(f"//a[@href='{item}']/text()")
            d1 = root.xpath(f"//a[@href='{item}']/time/@datetime")
            try:
                df.loc[len(df)+1] = [ext[0],r1,item,d1]
            except:
                df.loc[len(df)+1] = [ext[0],"Надо поработать со сторонними сайтами",item, "Дата"]
        else:
            df.loc[len(df)+1] = get_atr_lentaru(url+ item)

In [ ]:
get_news_mailru()
get_news_lentaru()

In [297]:
df.tail(10)


,source,news,linc,date
201,lenta.ru,Севший на мель у Одессы танкер связали с контр...,https://lenta.ru/news/2019/11/23/contrabanda/,2019-11-23T18:01:44+03:00
202,lenta.ru,Красное золото,https://lenta.ru/photo/2019/11/20/safran/,не определена
203,Коммерсантъ,В Петербурге открыли памятник Михаилу Калашникову,https://r.mail.ru/n314711141?&test_id=63&rnd=1...,2019-11-22T16:28:51+03:00
204,Фонтанка,Суд распустил избирком муниципалитета «Семенов...,https://r.mail.ru/n314716489?&test_id=63&rnd=1...,2019-11-22T18:13:03+03:00
205,АиФ Санкт-Петербург,В Петербурге теплая погода побила рекорд 1982 ...,https://r.mail.ru/n314710271?&test_id=63&rnd=1...,2019-11-22T16:17:22+03:00
206,ТАСС,Подозреваемых в сливе неизвестных веществ в ка...,https://r.mail.ru/n314763974?&test_id=63&rnd=1...,2019-11-23T14:23:08+03:00
207,ИА REGNUM,Женщина и ребёнок пострадали в ДТП с семью авт...,https://r.mail.ru/n314751829?&test_id=63&rnd=1...,2019-11-23T09:56:49+03:00
208,Новости,Новый внедорожник для миллиардеров: он похож н...,https://r.mail.ru/n314761702?&test_id=63&rnd=1...,2019-11-22T17:30:45+03:00
209,не подписано,3 законных способа отследить чужой смартфон,https://r.mail.ru/n314764030?&test_id=63&rnd=1...,2019-11-23T14:15:00+03:00
210,Анастасия Василевская,«Какая красивая Аллочка»: Галкин показал новое...,https://r.mail.ru/n314769752?&test_id=63&rnd=1...,2019-11-23T13:11:43+03:00
